Data to Use 📊
You will use the cleaned Pandas DataFrames, df_customers and df_transactions, resulting from the Day 1 exercise.

In [3]:
import pandas as pd
from datetime import datetime as dt

In [ ]:
# Load the CSV file into a DataFrame
try:
  customer_df = pd.read_csv("data/ex1-cleaned_customer_data.csv")
  display(customer_df.head())
except Exception as e:
  print(f"An error occurred while loading Customer Master Data.csv: {e}")

An error occurred while loading Customer Master Data.csv: [Errno 2] No such file or directory: 'data-engineer-interview-exercise/fraud_detection_exercise/ex1-cleaned_customer_data.csv'


In [ ]:
# Load the CSV file into a DataFrame
try:
  transaction_df = pd.read_csv("/data/ex1-cleaned_transaction_events.csv")
  display(transaction_df.head())
except Exception as e:
  print(f"An error occurred while loading Customer Master Data.csv: {e}")

,transaction_id,customer_id,amount,timestamp,currency,ip_address,transaction_hour
0,TX001,C101,150.75,2024-07-16 10:00:00,USD,192.168.1.10,10
1,TX002,C102,25.00,2024-07-16 10:01:30,USD,192.168.1.11,10
2,TX003,C101,50.25,2024-07-16 10:02:00,USD,192.168.1.10,10
3,TX004,C103,1200.00,2024-07-16 10:03:15,EUR,192.168.1.12,10
4,TX005,C102,75.50,2024-07-16 10:04:00,USD,192.168.1.11,10


In [ ]:
# Merge df_transactions with df_customers based on customer_id

try:
  df_enriched_transaction = pd.merge(transaction_df, customer_df, on='customer_id', how='left')
  display(df_enriched_transaction.head())
  display(df_enriched_transaction.info())
except Exception as e:
  print(f"An error occurred while joining the dataframes: {e}")

,transaction_id,customer_id,amount,timestamp,currency,ip_address,transaction_hour,customer_name,customer_email,registration_date,customer_tier,last_login_date
0,TX001,C101,150.75,2024-07-16 10:00:00,USD,192.168.1.10,10,Alice Smith,alice@example.com,2023-01-15,Gold,2024-07-15
1,TX002,C102,25.00,2024-07-16 10:01:30,USD,192.168.1.11,10,Bob Johnson,bob@example.com,2024-07-09,Silver,2024-07-16
2,TX003,C101,50.25,2024-07-16 10:02:00,USD,192.168.1.10,10,Alice Smith,alice@example.com,2023-01-15,Gold,2024-07-15
3,TX004,C103,1200.00,2024-07-16 10:03:15,EUR,192.168.1.12,10,Charlie Brown,charlie@example.com,2023-11-20,Bronze,2024-07-14
4,TX005,C102,75.50,2024-07-16 10:04:00,USD,192.168.1.11,10,Bob Johnson,bob@example.com,2024-07-09,Silver,2024-07-16


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     15 non-null     object 
 1   customer_id        15 non-null     object 
 2   amount             15 non-null     float64
 3   timestamp          15 non-null     object 
 4   currency           15 non-null     object 
 5   ip_address         15 non-null     object 
 6   transaction_hour   15 non-null     int64  
 7   customer_name      14 non-null     object 
 8   customer_email     14 non-null     object 
 9   registration_date  14 non-null     object 
 10  customer_tier      14 non-null     object 
 11  last_login_date    14 non-null     object 
dtypes: float64(1), int64(1), object(10)
memory usage: 1.5+ KB


None

In [ ]:
def is_fraudulent(registration_day, last_transaction_day, amount) -> bool:
  """
  Checks if a transaction is potentially fraudulent based on registration date and transaction amount.

  Args:
    registration_day: The registration date of the customer (string).
    last_transaction_day: The timestamp of the transaction (string).
    amount: The transaction amount (float).

  Returns:
    True if the transaction is potentially fraudulent, False otherwise.
  """
  try:
    last_transaction_day = pd.to_datetime(last_transaction_day)
    registration_day = pd.to_datetime(registration_day)
  except Exception as e:
    print(f"An error occurred while converting the dates: {e}")

  # Define "new customer" as someone registered within the last 7 days relative to the transaction date
  seven_days_ago = last_transaction_day - pd.Timedelta(days=7)

  is_new_customer = registration_day > seven_days_ago
  is_high_value = float(amount) > 500

  return is_new_customer and is_high_value

try:
  df_enriched_transaction['is_fraudulent_rule1'] = df_enriched_transaction.apply(lambda x: is_fraudulent(x['registration_date'], x['timestamp'], x['amount']), axis=1)
  display(df_enriched_transaction)
except Exception as e:
  print(f"An error occurred while applying the function: {e}")

,transaction_id,customer_id,amount,timestamp,currency,ip_address,transaction_hour,customer_name,customer_email,registration_date,customer_tier,last_login_date,is_fraudulent_rule1
0,TX001,C101,150.75,2024-07-16 10:00:00,USD,192.168.1.10,10,Alice Smith,alice@example.com,2023-01-15,Gold,2024-07-15,False
1,TX002,C102,25.00,2024-07-16 10:01:30,USD,192.168.1.11,10,Bob Johnson,bob@example.com,2024-07-09,Silver,2024-07-16,False
2,TX003,C101,50.25,2024-07-16 10:02:00,USD,192.168.1.10,10,Alice Smith,alice@example.com,2023-01-15,Gold,2024-07-15,False
3,TX004,C103,1200.00,2024-07-16 10:03:15,EUR,192.168.1.12,10,Charlie Brown,charlie@example.com,2023-11-20,Bronze,2024-07-14,False
4,TX005,C102,75.50,2024-07-16 10:04:00,USD,192.168.1.11,10,Bob Johnson,bob@example.com,2024-07-09,Silver,2024-07-16,False
5,TX006,C104,300.00,2024-07-16 10:05:00,USD,192.168.1.13,10,Diana Prince,diana@example.com,2024-07-10,Silver,2024-07-16,False
6,TX007,C101,10.00,2024-07-16 10:06:00,USD,192.168.1.14,10,Alice Smith,alice@example.com,2023-01-15,Gold,2024-07-15,False
7,TX008,C105,0.00,2024-07-16 10:07:00,USD,192.168.1.15,10,Eve Adams,eve@example.com,2023-05-01,Gold,2024-07-15,False
8,TX009,C102,200.00,2024-07-16 10:08:00,USD,192.168.1.11,10,Bob Johnson,bob@example.com,2024-07-09,Silver,2024-07-16,False
9,TX010,C106,45.00,2024-07-16 10:09:00,USD,192.168.1.16,10,Frank White,frank@example.com,2024-07-11,Bronze,2024-07-15,False


In [ ]:
# Count and print the number of transactions flagged by Rule 1
flagged_transactions_count = df_enriched_transaction['is_fraudulent_rule1'].sum()
print(f"Number of transactions flagged by Rule 1: {flagged_transactions_count}")

# Display the requested columns for the flagged transactions
flagged_transactions = df_enriched_transaction[df_enriched_transaction['is_fraudulent_rule1'] == True]
display(flagged_transactions[['transaction_id', 'customer_id', 'amount', 'timestamp', 'is_fraudulent_rule1']])

Number of transactions flagged by Rule 1: 0


,transaction_id,customer_id,amount,timestamp,is_fraudulent_rule1


In [ ]:
# Save the enriched transactions DataFrame to a CSV file
try:
    df_enriched_transaction.to_csv('df_enriched_transaction.csv', index=False)
    print("df_enriched_transaction saved successfully to df_enriched_transaction.csv")
except Exception as e:
    print(f"An error occurred while saving df_enriched_transaction: {e}")

df_enriched_transaction saved successfully to df_enriched_transaction.csv


1. **Extraction: How would you conceptually extract the df_enriched_transactions data for loading?**
> It depends on their format and how they are stored. When I know those details, I can use ad hoc function of my ETL framework to load the dataset. For instance we can use `pd.read_csv()` method or in Spark we can use `spark_df = spark.read.csv('your_file.csv', header=True, inferSchema=True)`

2. **Transformation (dim_customer): How would you manage new vs. existing customers in the dim_customer table (e.g., INSERT vs. UPDATE)? What SQL operations would be needed?**
> Add new customer/s: if I want to add only one new record in the customer table, assuming that we are working with SQL, I can use the command `INSERT INTO table_name` and then i can list the values, for instance:  
```
  INSERT INTO table_name (column1, column2, column3, ...)
  VALUES (value1, value2, value3, ...);
```
Otherwise is we have a bulk of records coming from another table or from a csv file we can use the `BULK INSERT` or `COPY` command

3. **Transformation (fact_transactions): How would you look up the customer_sk from dim_customer? What SQL operations would be required for this table?**
> I need to join the dim_customer table and the fact_transactions table on the customer_sk value using `JOIN dim_customer fact_transactions ON fact_transactions.customer_sk = dim_customer.customer_sk`. The key customer_sk is a surrogate key that we use for multiple reasons as the primary key in the dim_customer table:
  - because we have our own primary key that is not dependent from external data, so we can update customer_id value
  - the surrogate key is usually a INT or something small, so it takes less time to do operations on top of it
  - we can use it to do updates of the records (SCD2 algo) and keep track of the history

4. **Transformation (fact_fraud_events): How would you select only the flagged transactions to populate this table, including the rule_id and detection_timestamp? How would you conceptually load the transformed data into the target tables?**
> To select the marked records from  `fact_transactions` and then insert them into `fact_fraud_events`, I have to create a query that operates in two ways:
first, we have to create the logic to select the columns we are interested into from `fact_transactions` and apply the command `WHERE is_fraudulent_rule1 = TRUE` to filter them; second, we use the INSERT INTO command to insert the values into the desired columns. So to recap: 1. think about the destination schema and write it down after the INSER INTO command; write a SELECT command specifing with the same order what values to put into the schema (if values are missing we can add them as default values, like a strin 'Rule 1' for rule_id); then remember to apply the WHERE command to filter the values from the table.
```
  INSERT INTO fact_fraud_events (
      fraud_event_sk,        -- Generated surrogate key for the fraud event
      transaction_sk,        -- Link to the transaction in fact_transactions
      rule_id,               -- The rule that flagged it (e.g., 'Rule 1')
      detection_timestamp,   -- When the fraud was detected
      severity               -- Severity level
  )
  SELECT
      -- Logic to generate fraud_event_sk (e.g., nextval('fraud_event_sk_sequence'))
      nextval('fraud_event_sk_sequence'), -- Example
      transaction_sk,        -- From the flagged transaction in fact_transactions
      'Rule 1',              -- Hardcoded or determined rule ID
      NOW(),                 -- Current timestamp or ETL process timestamp
      'High'                 -- Determined severity
  FROM
      fact_transactions
  WHERE
      is_fraudulent_rule1 = TRUE;
```


5. **Edge Case 1: Referential Integrity Violation** ⚠️: A `customer_id` in `df_transactions` does not exist in `df_customers` (e.g., 'C999' from the sample data). Describe how a robust ETL pipeline should handle this (e.g., left join and identify unmatched, quarantine data, or use a default customer record).
> There are several ways to maange this kind of mistake: we can do a left join and consider to have some records without customer information, we can then filter those records and insert them into a quarantine table to run another ETL pipeline which can fix or discard the records; we can fill the missing values with standard values and register all of them in the main table, we will run some other process to improve the quality of this data, this maintain the integrity of the schema and does not create a new table in the database to handle. We can choose to perform a mix of these two methods based on our requirements.

6. **Edge Case 2: Slowly Changing Dimensions (SCD Type 2)** ⏳: Imagine a customer's `customer_tier` can change, and you must preserve the historical tier for past transactions. Explain conceptually how you would modify the `dim_customer` table and the ETL process to support SCD Type 2 (e.g., by adding `start_date`, `end_date`, and `is_current` columns).
> we can add new columns to the table to sustain the SCD 2 algorithm such as `start_date`, `end_date`, `is_current`. We have to run a "deactivation" process for the old records and insert process for the new one. Whenever we start our ingesting pipeline we run:
```
UPDATE dim_customer
SET
    end_date = @change_detection_timestamp, --when the ingestion started
    is_current = FALSE
WHERE
    customer_id = @changing_customer_id -- these are the incoming customer ids
    AND is_current = TRUE, -- so that we update the last "active" record with that customer id
```
Then we have to run a INSERT command setting `start_date` with the ingestion timestamp, `end_date` will be the upper limit for this variable such as "31.12.9999" and `is_current` will be set to True.
```
INSERT INTO dim_customer (...)
SELECT
    nextval('dim_customer_sk_sequence'), -- Generate a new surrogate key
    @changing_customer_id,
    ...
    ...
    registration_date,   -- Get the original registration date (assuming it doesn't change)
    @new_customer_tier, -- if they change their tier
    ...
    @change_detection_timestamp, -- The start date of this new version
    '9999-12-31',        -- A far future date indicating this is currently valid
    TRUE                 -- Mark as the current record
FROM
    dim_customer         -- Select from dim_customer to get unchanged attributes like registration_date
WHERE
    customer_id = @changing_customer_id
    AND is_current = FALSE -- Select the *just deactivated* record to get static attributes if needed
LIMIT 1; -- Ensure you only get one record if multiple were somehow marked as is_current=TRUE
```
